In [1]:
import tensorflow as tf
import gym

from rltensor.agents import DQN
from rltensor.processors import AtariProcessor
from rltensor.executions import Runner
from rltensor.configs import dqn_config, fit_config
from rltensor.environments import RepeatEnvWrapper

env_name = 'Breakout-v0'
env = gym.make(env_name)
repeat_actions = 4
env = RepeatEnvWrapper(env, repeat_actions)

conf = dict(
    action_spec={"type": "int", "shape": env.action_space.n},
)
default_config = dqn_config()
conf.update(default_config)

_fit_config = fit_config()
# _fit_config["log_freq"] = 10
# conf["t_learn_start"] = 10
conf["deep_q"] = False
save_file_path = "./params/model.ckpt"
    
tf.reset_default_graph()
dqn = DQN(**conf)
runner = Runner(agent=dqn, env=env, env_name=env_name,
                tensorboard_dir="./logs")
runner.fit(save_video_path=None, **_fit_config)

/home/tomoaki/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
[2017-12-22 06:52:19,554] Making new env: Breakout-v0


Building tensorflow graph...
Finished building tensorflow graph, spent time: 0.2579231262207031


  0%|          | 16/50000000 [00:00<88:39:18, 156.66it/s]

Model saved in file: params/model.ckpt


  0%|          | 60735/50000000 [04:42<161:20:24, 85.98it/s] 

Model saved in file: params/model.ckpt


In [18]:
load_file_path = "./params/model.ckpt"
# runner = Runner(agent=dqn, env=env, env_name=env_name,
#                 load_file_path=load_file_path, tensorboard_dir="./test_logs")
runner.play(5, render_freq=1, ep=.05, log_freq=10)

In [ ]:
save_file_path = "./params/model.ckpt"
# runner = Runner(agent=dqn, env=env, env_name=env_name, is_initialize=False,
#                 load_file_path=load_file_path, tensorboard_dir="./logs")
runner.fit(save_video_path=None, **_fit_config)


  0%|          | 4/49349831 [00:00<345:27:54, 39.68it/s]

Model saved in file: params/model.ckpt



  0%|          | 16183/49349831 [03:11<163:30:43, 83.81it/s]